# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [ ]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [ ]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [ ]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [ ]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [ ]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Find what features had non-zero weight.

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [ ]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [ ]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [ ]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [ ]:
l1_penalty_min = 
l1_penalty_max = 

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [ ]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [7]:
def get_residual_sum_of_squares(model, data, outcome):
    RSS = None
    data['prediction'] = model.predict(data)
    data['error'] = outcome - data['prediction']
    error_sq = data['error'] * data['error']
    RSS = error_sq.sum()
    return(RSS)

import graphlab

from math import log, sqrt

import sys

import numpy as np

sales = graphlab.SFrame('kc_house_data.gl/')

sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

# -------------------------------------
# Learn regression weights with L1 penalty
# -------------------------------------
print("*** Learn regression weights with L1 penalty")
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10,
                                              verbose=False)

coef = model_all.get("coefficients")
print coef.print_rows(num_rows=18, num_columns=4)
print coef[coef["value"]!=0]
print "Number of non-zero coefficients: %d" % len(coef[coef["value"]!=0])

# -------------------------------------
# Selecting an L1 penalty
# -------------------------------------
print("*** Selecting an L1 penalty")
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

result = []
lowest_rss = None
best_l1_penalty = None
best_model = None
for l1_penalty in np.logspace(1,7,num=13):
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty,
                                              verbose=False)
    rss = get_residual_sum_of_squares(model, validation, validation['price'])
    print("l1_penalty = %f, rss = %f" % (l1_penalty, rss))
    result.append("l1_penalty = %f, rss = %f" % (l1_penalty, rss))
    if lowest_rss is None:
        lowest_rss = rss
        best_l1_penalty = l1_penalty
        best_model = model
    if rss < lowest_rss:
        lowest_rss = rss
        best_l1_penalty = l1_penalty
        best_model = model

for r in result:
    print(r)

print "best l1_penalty = %f" % best_l1_penalty

rss = get_residual_sum_of_squares(best_model, testing, testing['price'])

print("testing: l1_penalty = %f, rss = %f" % (best_l1_penalty, rss))
coef = best_model.get("coefficients")
coef.print_rows(num_rows=18, num_columns=4)
nonzero_coef = coef[coef["value"]!=0]
print "Number of non-zero coefficients: %d" % len(nonzero_coef)
print nonzero_coef

# -------------------------------------
# Limit the number of nonzero weights
# -------------------------------------
print("*** Limit the number of nonzero weights")

max_nonzeros = 7

l1_penalty_min = None
l1_penalty_max = None
l1_penalty_values = np.logspace(8, 10, num=20)
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty,
                                              verbose=False)
    coef = model.get("coefficients")
#    print coef[coef["value"]!=0]
    nonzero_coef = coef[coef["value"]!=0]
    print("L1 penalty: %f" % l1_penalty)
    print("  Number of non-zero coefficients: %d" % len(nonzero_coef["value"]))

    if len(nonzero_coef["value"]) > max_nonzeros:
        l1_penalty_min = l1_penalty

    if l1_penalty_max is None and len(nonzero_coef["value"]) < max_nonzeros:
        l1_penalty_max = l1_penalty

print("Max L1 penalty: %f" % l1_penalty_max)
print("Min L1 penalty: %f" % l1_penalty_min)

# -------------------------------------
# Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set
# -------------------------------------
print("*** Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set")

l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

result = []
lowest_rss = None
best_l1_penalty = None
best_model = None
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty, verbose=False)

    coef = model.get("coefficients")
#    print coef[coef["value"]!=0]
    nonzero_coef = coef[coef["value"]!=0]
    print("L1 penalty: %f" % l1_penalty)
    print("  Number of non-zero coefficients: %d" % len(nonzero_coef["value"]))
    sparsity = len(coef["value"]) - len(nonzero_coef["value"])
    print("  Sparsity: %d" % sparsity)

    if len(nonzero_coef["value"]) == max_nonzeros:
        rss = get_residual_sum_of_squares(model, validation, validation['price'])
        print("  Validation RSS = %f" % rss)
        if lowest_rss is None:
            lowest_rss = rss
            best_l1_penalty = l1_penalty
            best_model = model
        if rss < lowest_rss:
            lowest_rss = rss
            best_l1_penalty = l1_penalty
            best_model = model

for r in result:
    print(r)

print "Best L1 penalty = %f" % best_l1_penalty
print "RSS = %f" % lowest_rss

rss = get_residual_sum_of_squares(best_model, testing, testing['price'])

print("testing: l1_penalty = %f, rss = %f" % (best_l1_penalty, rss))
coef = best_model.get("coefficients")
print coef[coef["value"]!=0]

*** Learn regression weights with L1 penalty
+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.02472

In [5]:
coef.print_rows(num_rows=18, num_columns=4)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
|  sqft_basement   |  None |

In [8]:
print("*** Limit the number of nonzero weights")

max_nonzeros = 7

l1_penalty_min = None
l1_penalty_max = None
l1_penalty_values = np.logspace(8, 10, num=20)
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty,
                                              verbose=False)
    coef = model.get("coefficients")
#    print coef[coef["value"]!=0]
    nonzero_coef = coef[coef["value"]!=0]
    print("L1 penalty: %f" % l1_penalty)
    print("  Number of non-zero coefficients: %d" % len(nonzero_coef["value"]))

    if len(nonzero_coef["value"]) > max_nonzeros:
        l1_penalty_min = l1_penalty

    if l1_penalty_max is None and len(nonzero_coef["value"]) < max_nonzeros:
        l1_penalty_max = l1_penalty

print("Max L1 penalty: %f" % l1_penalty_max)
print("Min L1 penalty: %f" % l1_penalty_min)

*** Limit the number of nonzero weights
L1 penalty: 100000000.000000
  Number of non-zero coefficients: 18
L1 penalty: 127427498.570313
  Number of non-zero coefficients: 18
L1 penalty: 162377673.918872
  Number of non-zero coefficients: 18
L1 penalty: 206913808.111479
  Number of non-zero coefficients: 18
L1 penalty: 263665089.873036
  Number of non-zero coefficients: 17
L1 penalty: 335981828.628379
  Number of non-zero coefficients: 17
L1 penalty: 428133239.871940
  Number of non-zero coefficients: 17
L1 penalty: 545559478.116851
  Number of non-zero coefficients: 17
L1 penalty: 695192796.177559
  Number of non-zero coefficients: 17
L1 penalty: 885866790.410083
  Number of non-zero coefficients: 16
L1 penalty: 1128837891.684688
  Number of non-zero coefficients: 15
L1 penalty: 1438449888.287666
  Number of non-zero coefficients: 15
L1 penalty: 1832980710.832438
  Number of non-zero coefficients: 13
L1 penalty: 2335721469.090121
  Number of non-zero coefficients: 12
L1 penalty: 297635

In [9]:
print("*** Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set")

l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

result = []
lowest_rss = None
best_l1_penalty = None
best_model = None
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty, verbose=False)

    coef = model.get("coefficients")
#    print coef[coef["value"]!=0]
    nonzero_coef = coef[coef["value"]!=0]
    print("L1 penalty: %f" % l1_penalty)
    print("  Number of non-zero coefficients: %d" % len(nonzero_coef["value"]))
    sparsity = len(coef["value"]) - len(nonzero_coef["value"])
    print("  Sparsity: %d" % sparsity)

    if len(nonzero_coef["value"]) == max_nonzeros:
        rss = get_residual_sum_of_squares(model, validation, validation['price'])
        print("  Validation RSS = %f" % rss)
        if lowest_rss is None:
            lowest_rss = rss
            best_l1_penalty = l1_penalty
            best_model = model
        if rss < lowest_rss:
            lowest_rss = rss
            best_l1_penalty = l1_penalty
            best_model = model

for r in result:
    print(r)

print "Best L1 penalty = %f" % best_l1_penalty
print "RSS = %f" % lowest_rss

rss = get_residual_sum_of_squares(best_model, testing, testing['price'])

print("testing: l1_penalty = %f, rss = %f" % (best_l1_penalty, rss))
coef = best_model.get("coefficients")
print coef[coef["value"]!=0]

*** Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set
L1 penalty: 2976351441.631313
  Number of non-zero coefficients: 10
  Sparsity: 8
L1 penalty: 3019316638.952415
  Number of non-zero coefficients: 10
  Sparsity: 8
L1 penalty: 3062281836.273518
  Number of non-zero coefficients: 10
  Sparsity: 8
L1 penalty: 3105247033.594620
  Number of non-zero coefficients: 10
  Sparsity: 8
L1 penalty: 3148212230.915722
  Number of non-zero coefficients: 10
  Sparsity: 8
L1 penalty: 3191177428.236824
  Number of non-zero coefficients: 10
  Sparsity: 8
L1 penalty: 3234142625.557926
  Number of non-zero coefficients: 10
  Sparsity: 8
L1 penalty: 3277107822.879028
  Number of non-zero coefficients: 10
  Sparsity: 8
L1 penalty: 3320073020.200130
  Number of non-zero coefficients: 8
  Sparsity: 10
L1 penalty: 3363038217.521233
  Number of non-zero coefficients: 8
  Sparsity: 10
L1 penalty: 3406003414.842335
  Number of